In [64]:
import requests

In [65]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [66]:
data = requests.get(standings_url)

In [67]:
from bs4 import BeautifulSoup

In [68]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

IndexError: list index out of range

In [10]:
team_url = [f"https://fbref.com{l}" for l in links]

In [13]:
data = requests.get(team_url[0])

In [14]:
import pandas as pd
matches =pd.read_html(data.text, match="Scores & Fixtures")

/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_967/1536848166.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches =pd.read_html(data.text, match="Scores & Fixtures")


In [16]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting' in l ]

In [21]:
data = requests.get(f"https://fbref.com{links[0]}")

In [22]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_967/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [23]:
shooting.head() 

For Liverpool                                                              \
           Date   Time            Comp         Round  Day Venue Result   GF   
0    2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1    2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2    2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3    2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4    2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   

                         ... Standard               Expected               \
    GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  0.0     Ipswich Town  ...     14.8  0.0  0     0      2.6  2.6    0.15   
1  0.0        Brentford  ...     13.6  1.0  0     0      2.5  2.5    0.14   
2  0.0   Manchester Utd  ...     13.4  0.0  0     0      1.8  1.8    0.16   
3  1.0  Nott'ham Forest  ...     14.9  0.0  0     0      0.9  0.9    0.07   
4  1.0         it Milan  ...     15.7  1.0  0     0      3.1  3.1    0.14   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -0.6    -0.6        Match Report  
1 -0.5    -0.5        Match Report  
2  1.2     1.2        Match Report  
3 -0.9    -0.9        Match Report  
4 -0.1    -0.1        Match Report  

[5 rows x 26 columns]

In [24]:
shooting.columns = shooting.columns.droplevel()

In [25]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [27]:
years = list(range(2024, 2020, -1))

In [29]:
all_matches =[]

In [30]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [31]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)


/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_967/1338002569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_967/1338002569.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_967/1338002569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_9

ValueError: No tables found

In [ ]:
len(all_matches)

In [ ]:
match_df=pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("EPLmathces.csv")